In [1]:
import torch

torch.cuda.is_available()

torch.cuda.device_count()


torch.cuda.current_device()


torch.cuda.device(0)


torch.cuda.get_device_name(0)


'NVIDIA GeForce RTX 3060 Laptop GPU'

In [7]:
import os
import torch
# import torchvision
from torchvision import datasets, models, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# Set device to GPU if available
device = torch.device("cuda")

# Define paths
train_dir = 'preprocessed_data_test'
# val_dir = 'Validation'

# Define image transformations for training and validation
train_transforms = transforms.Compose([
    # transforms.RandomResizedCrop(224),
    transforms.Resize((224, 224)),  # Resizing the image to 224x224
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# val_transforms = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
# val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

num_classes = 39  # Change this to the number of classes in your dataset

# Initialize the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers of the base model
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes)
)


# Load the model state from the 10th epoch
model_path = 'model/last/plant_disease_detection_resnet50_finetuned.pth'
model.load_state_dict(torch.load(model_path))

model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

def train(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

# Resume training from the 11th epoch
start_epoch = 1
num_epochs = 200
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(start_epoch, num_epochs+1):
    print(f"Epoch {epoch}/{num_epochs}")

    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    # val_loss, val_acc = validate(model, val_loader, criterion)

    train_losses.append(train_loss)
    # val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    # val_accuracies.append(val_acc)

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    torch.save(model.state_dict(), f'model/last/plant_disease_detection_resnet50_finetuned{epoch}.pth')

    # print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

# Fine-tune by unfreezing some layers of the base model
for param in model.layer4.parameters():
    param.requires_grad = True

# Lower the learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Fine-tune the model
fine_tune_epochs = 200
start_fine_tune_epoch = 1  # If fine-tuning from scratch, else adjust this value accordingly
for epoch in range(start_fine_tune_epoch, fine_tune_epochs+1):
    print(f"Fine-Tuning Epoch {epoch}/{fine_tune_epochs}")

    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    # val_loss, val_acc = validate(model, val_loader, criterion)

    train_losses.append(train_loss)
    # val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    # val_accuracies.append(val_acc)

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    torch.save(model.state_dict(), f'models/plant_disease_detection_resnet50_finetuned_{epoch}.pth')
    torch.save(model.state_dict(), f'models/plant_disease_detection_resnet50_finetuned_{epoch}_model.pth')

    # print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')

# Final model save
torch.save(model.state_dict(), 'plant_disease_detection_resnet50_finetuned_final.pth')

torch.save(model,"plant_disease_detection_resnet50_finetuned_final_model.pth")

/tmp/ipykernel_46042/3685769303.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Epoch 11/20


Train Loss: 0.5356, Train Accuracy: 83.80%
Epoch 12/20


Train Loss: 0.5118, Train Accuracy: 84.13%
Epoch 13/20


Train Loss: 0.5006, Train Accuracy: 84.44%
Epoch 14/20


...

Train Loss: 0.4792, Train Accuracy: 85.04%
Epoch 18/20


Train Loss: 0.4753, Train Accuracy: 85.14%
Epoch 19/20


Train Loss: 0.4665, Train Accuracy: 85.55%
Epoch 20/20


Train Loss: 0.4696, Train Accuracy: 85.38%
Fine-Tuning Epoch 1/20


KeyboardInterrupt: 

In [ ]:

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:

import os
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# Set device to GPU if available
device = torch.device("cuda")

# Define paths
train_dir = 'preprocessed_data_test'
# val_dir = 'Validation'

# Define image transformations for training and validation
train_transforms = transforms.Compose([
    # transforms.RandomResizedCrop(224),
    transforms.Resize((224, 224)),  # Resizing the image to 224x224
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# val_transforms = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
# val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

num_classes = 39  # Change this to the number of classes in your dataset

# Initialize the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []


def train(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc


# Freeze all layers of the base model
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes)
)


# Load the model state from the 10th epoch
model_path = 'model/last/plant_disease_detection_resnet50_finetuned20.pth'
model.load_state_dict(torch.load(model_path))

model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)




# Fine-tune by unfreezing some layers of the base model
for param in model.layer4.parameters():
    param.requires_grad = True

# Lower the learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Fine-tune the model
fine_tune_epochs = 20
start_fine_tune_epoch = 1  # If fine-tuning from scratch, else adjust this value accordingly
for epoch in range(start_fine_tune_epoch, fine_tune_epochs+1):
    print(f"Fine-Tuning Epoch {epoch}/{fine_tune_epochs}")

    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    # val_loss, val_acc = validate(model, val_loader, criterion)

    train_losses.append(train_loss)
    # val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    # val_accuracies.append(val_acc)

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    torch.save(model.state_dict(), f'model/full/plant_disease_detection_resnet50_finetuned{epoch}.pth')

    # print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')

# Final model save
torch.save(model.state_dict(), 'plant_disease_detection_resnet50_finetuned_final.pth')


/tmp/ipykernel_83783/1696265683.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Fine-Tuning Epoch 1/20


KeyboardInterrupt: 

In [ ]:
num_classes = 39

# Initialize the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Replace the final fully connected layer
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes)
)

In [ ]:
# Freeze all layers of the base model
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes)
)

# Define optimizer and loss function for the new layers
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Train the model
for epoch in range(start_epoch, num_epochs+1):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    print(f"Epoch {epoch} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    torch.save(model.state_dict(), f'models/plant_disease_detection_resnet50_finetuned_{epoch}.pth')


In [ ]:
# Unfreeze the layer4 block for fine-tuning
for param in model.layer4.parameters():
    param.requires_grad = True

# Lower the learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Fine-tune the model
for epoch in range(start_fine_tune_epoch, fine_tune_epochs+1):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    print(f"Fine-Tuning Epoch {epoch} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    torch.save(model.state_dict(), f'models/plant_disease_detection_resnet50_finetuned_{epoch}.pth')
